In [1]:
from seed_benchmarks import build_chain, run_seeds
from faker import Faker
import random
import pandas as pd
from langchain import PromptTemplate

## We analyse numbers, big numbers, and words as potential seeds

In [14]:
no_seed = [0] * 10

number_seeds = range(10)

big_number_seeds = [random.randint(1000000000, 9999999999) for _ in range(10)]
print(big_number_seeds)

faker = Faker()
word_seeds = [faker.word() for _ in range(10)]
print(word_seeds)

seed_types = {#"num" : number_seeds, 
              #"big_num" : big_number_seeds, 
              "words" : word_seeds}

[6888445919, 1610077256, 5763368075, 1009933493, 6632799828, 2669850140, 5316075611, 6096929598, 6364342291, 6696683901]
['thing', 'charge', 'show', 'audience', 'myself', 'beyond', 'along', 'pass', 'probably', 'state']


## We will try various templates to combine with the seeds

In [15]:
mood_prompt = PromptTemplate(
    input_variables=["seed"],
    template="""
    You answer in character where your mood is represented by: {seed}

    Give me one baby name. Only output the name, no explanation or other text
    """,
)

mood_chain = build_chain(mood_prompt)

explain_prompt = PromptTemplate(
    input_variables=["seed"],
    template="""
    You are using <{seed}> to help you generate a random response to the following prompt:

    Give me one baby name. Only output the name, no explanation or other text
    """,
)

explain_chain = build_chain(explain_prompt)


set_seed_prompt = PromptTemplate(
    input_variables=["seed"],
    template="""
    Set seed: {seed}

    Give me one baby name. Only output the name, no explanation or other text
    """,
)

set_seed_chain = build_chain(set_seed_prompt)

inspired_prompt = PromptTemplate(
    input_variables=["seed"],
    template="""
    Be inspired by this word in your response: {seed}

    Give me one baby name. Only output the name, no explanation or other text
    """,
)

inspired_chain = build_chain(inspired_prompt)

raw_seed_prompt = PromptTemplate(
    input_variables=["seed"],
    template="""
    {seed}

    Give me one baby name. Only output the name, no explanation or other text
    """,
)

raw_seed_chain = build_chain(raw_seed_prompt)

chains = {"mood" : mood_chain, 
          #"explain" : explain_chain,
          #"set_seed" : set_seed_chain, 
          "inspire" : inspired_chain, 
          #"raw" : raw_seed_chain
         }

In [16]:
summary_df = pd.DataFrame({'no_change_in_seed' : run_seeds(raw_seed_chain, no_seed)})

In [17]:
for seed_name, seed_value in seed_types.items():
    for chain_name, chain_value in chains.items():
        output_list = run_seeds(chain_value, seed_value)
        output_name = f"{chain_name} {seed_name}"
        summary_df[output_name] = output_list
        

## Running all combinations yields the most promising combinations

In [18]:
summary_df

,no_change_in_seed,mood words,inspire words
0,Avery,Luna.,Avery
1,Avery,"""Maximus""",Avery
2,Avery,*smiling brightly* \n\nAvery.,Avery
3,Avery,"Audience, I present to you the name ""Evelyn"".",Audience\n\nAvery
4,Avery,Liam.,Avery
5,Avery,Aurora.,Aria
6,Avery,"Along: ""Aria""",Avery
7,Avery,Pass.,Pass\n\nAvery
8,Avery,"Probably, I suggest the name ""Aria"".",Avery
9,Avery,Emma.,Oliver
